In [4]:
#!pip install jsonpickle

In [5]:
import flask
from flask import request, jsonify
import jsonpickle

import pandas as pd
import numpy as np

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, PowerTransformer, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import pickle

In [7]:
with open('model.pickle', 'rb') as f:
    model = pickle.load(f)
    
#model

In [8]:
with open('rc.pickle', 'rb') as f:
    rc = pickle.load(f)
    
#rc

In [9]:
def row_process(route_category):
    if route_category not in rc.keys():
        return route_category
    
    if rc[route_category] > 3:  #10
        return route_category
    
    return 'other'

def preprocess(df_p):
    df_p['route'] = df_p['st_code_snd'] + df_p['st_code_rsv']
    df_p['route_category'] = df_p['route'].apply(row_process)

In [10]:
features = ['route_category', 'date_depart_week', 'date_depart_month',
            'route_type', 'is_load', 'rod', 'common_ch', 'vidsobst', 'distance', 'date_depart_hour',
           'dayofweek', 'snd_org_id', 'rsv_org_id', 'snd_roadid', 
            'rsv_roadid',  # ну можно и не убирать
            'snd_dp_id', 'rsv_dp_id']

In [11]:
def predict_delivery_time(df):
    
    preprocess(df)
    
    df['year'] = df['date_depart_year']
    df['month'] = df['date_depart_month']
    df['day'] = df['date_depart_day']
    df['hour'] = df['date_depart_hour']
    df['minute'] = 0
    df['second'] = 0
    df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour', 'minute', 'second']])
    df['dayofweek'] = df['datetime'].dt.dayofweek
    
    predict = model.predict(df[features])
    
    return predict[0]

In [12]:
parameters = \
    {
        'st_code_snd':['', 'str'],
        'st_code_rsv':['', 'str'],
        'date_depart_year':[0, 'int64'],
        'date_depart_month':[0, 'int64'],
        'date_depart_week':[0, 'int64'],
        'date_depart_day':[0, 'int64'],
        'date_depart_hour':[0, 'int64'],
        'fr_id':[0, 'float64'],
        'route_type':[0, 'float64'],
        'is_load':[0, 'int64'],
        'rod':[0, 'int64'],
        'common_ch':[0, 'float64'],
        'vidsobst':[0, 'float64'],
        'distance':[0, 'float64'],
        'snd_org_id':[0, 'int64'],
        'rsv_org_id':[0, 'int64'],
        'snd_roadid':[0, 'int32'],
        'rsv_roadid':[0, 'int32'],
        'snd_dp_id':[0, 'int64'], 
        'rsv_dp_id':[0, 'int64']}
data = {}

In [13]:
app = flask.Flask(__name__)
app.config["DEBUG"] = True

In [14]:
@app.route('/predict_delivery_time', methods=['GET'])
def get_predict_delivery_time():
    
    for n in parameters.keys():
        if n in request.args:
            data[n] = [str(request.args[n])]
        else:
            data[n] = [parameters[n][0]]
    
    df = pd.DataFrame(data)
    for n in parameters.keys():
        df[n] = df[n].astype(parameters[n][1])
    
    delivery_time = predict_delivery_time(df)
    #return jsonpickle.encode(delivery_time)
    return str(delivery_time)

In [ ]:
if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 5000, app)
    
#app.run(host='0.0.0.0', port=5000)

 * Running on http://localhost:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2022 02:49:07] "GET /predict_delivery_time?st_code_snd=happy:)pgkwagon:):)&st_code_rsv=pgk2022newyear:)pgk&date_depart_year=2022&date_depart_month=9&date_depart_week=35&date_depart_day=1&date_depart_hour=9&fr_id=2261.0&route_type=3.0&is_load=0&rod=2&common_ch=10.0&vidsobst=102.0&distance=2930.0&snd_org_id=2974&rsv_org_id=2847&snd_roadid=28&rsv_roadid=28&snd_dp_id=134&rsv_dp_id=125 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 02:49:16] "GET /predict_delivery_time?st_code_snd=happy%3A%29pgkwagon%3A%29%3A%29&st_code_rsv=pgk2022newyear%3A%29pgk&date_depart_year=2022&date_depart_month=9&date_depart_week=35&date_depart_day=1&date_depart_hour=9&fr_id=2261.0&is_load=0&rod=2&common_ch=10.0&vidsobst=102.0&distance=2930.0&snd_org_id=2974&rsv_org_id=2847 HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2022 02:49:29] "GET /predict_delivery_time?st_code_snd=happy%3A%29pgkwagon%3A%29%3A%29&st_code_rsv=pgk2022newyear%3A%29pgk&date_d